In [ ]:
from typing import *

from pyk4a import PyK4A, Config
from matplotlib import pyplot as plt
import numpy as np
import cv2


device_id = 0
config = {}


# Load camera with the default config
config = Config(**config)
print(config.unpack())


k4a = PyK4A(device_id=device_id, config=config)
k4a.start()

In [ ]:
def colorize(
    image: np.ndarray,
    clipping_range: Tuple[Optional[int], Optional[int]] = (None, None),
    colormap: int = cv2.COLORMAP_HSV,
) -> np.ndarray:
    if clipping_range[0] or clipping_range[1]:
        img = image.clip(clipping_range[0], clipping_range[1])  # type: ignore
    else:
        img = image.copy()
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img = cv2.applyColorMap(img, colormap)
    return img

In [ ]:
# Get the next capture (blocking function)
capture = k4a.get_capture()

In [ ]:
color, transformed_depth = capture.color, capture.transformed_depth
transformed_color, depth = capture.transformed_color, capture.depth
color = color[:, :, 2::-1] # BGRA to RGB
transformed_color = transformed_color[:, :, 2::-1] # BGRA to RGB

depth_coord = capture.depth_point_cloud / 1000.
color_coord = capture.transformed_depth_point_cloud / 1000.

plt.figure(figsize = (20,3))
plt.imshow(np.concatenate([color, colorize(transformed_depth)], axis=-2))
plt.show()

In [ ]:
import open3d as o3d

In [ ]:
depth_frame = True

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector((depth_coord if depth_frame else color_coord).reshape(-1,3))
pcd.colors = o3d.utility.Vector3dVector((transformed_color if depth_frame else color).reshape(-1,3)/255.)

In [ ]:
o3d.visualization.draw_geometries([pcd])